In [55]:
import os
from typing import List

import requests as requests
from typing import Union

In [68]:
def alchemy_asset_transfers(
        from_address: str,
        to_address: str,
        from_block: Union[int, str],
        to_block: Union[int, str] = 'latest',
        page_key: str = None
):
    if not to_block == 'latest':
        to_block = hex(to_block)

    parameters = {
        "fromBlock": hex(from_block),
        "toBlock": to_block,
        "fromAddress": from_address,
        "toAddress": to_address
    }

    if not from_address:
        parameters.pop('fromAddress')
    if not to_address:
        parameters.pop('toAddress')

    if page_key:
        parameters['pageKey'] = page_key

    response_post = requests.post(
        "https://eth-mainnet.alchemyapi.io/v2/"
        + os.environ["ALCHEMY_API_KEY"],
        json={
            "jsonrpc": "2.0",
            "id": 0,
            "method": "alchemy_getAssetTransfers",
            "params": [parameters]
        },
    )

    response_json = response_post.json()
    return response_json

def get_lp_swaps(
        from_address: str,
        to_address: str,
        from_block: Union[int, str],
        to_block: Union[int, str]
) -> List:

    page_key = None
    all_txes = []
    while True:
        response = alchemy_asset_transfers(
            from_address=from_address,
            to_address=to_address,
            from_block=from_block,
            to_block=to_block,
            page_key=page_key
        )
        transactions = response["result"]["transfers"]
        all_txes.extend(transactions)
        if not 'pageKey' in response['result'].keys():
            break
        page_key = response['result']['pageKey']

    return all_txes

In [69]:
block_start = 13483245
pool_addr = "0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7"

In [70]:
txes = get_lp_swaps(
    from_address=pool_addr, to_address='', from_block=block_start, to_block='latest'
)

In [71]:
txes

[{'blockNum': '0xcdbd14',
  'hash': '0xdc1f480f3e470bb22dfbe38f5399abf91c05db6b8eb2dd42bac0b9889d061bf2',
  'from': '0xbebc44782c7db0a1a60cb6fe97d0b483032ff1c7',
  'to': '0xed279fdd11ca84beef15af5d39bb4d4bee23f0ca',
  'value': 414375.4527277479,
  'erc721TokenId': None,
  'erc1155Metadata': None,
  'asset': 'DAI',
  'category': 'token',
  'rawContract': {'value': '0x57bf569b10a52aad70e3',
   'address': '0x6b175474e89094c44da98b954eedeac495271d0f',
   'decimal': '0x12'}},
 {'blockNum': '0xcdbd15',
  'hash': '0x410a77a44b532ee42a5a50173f848d537c32301520da8a90242905b34ea8ab4c',
  'from': '0xbebc44782c7db0a1a60cb6fe97d0b483032ff1c7',
  'to': '0xed279fdd11ca84beef15af5d39bb4d4bee23f0ca',
  'value': 424332.8130128671,
  'erc721TokenId': None,
  'erc1155Metadata': None,
  'asset': 'DAI',
  'category': 'token',
  'rawContract': {'value': '0x59db20bd279af34abdf2',
   'address': '0x6b175474e89094c44da98b954eedeac495271d0f',
   'decimal': '0x12'}},
 {'blockNum': '0xcdbd1c',
  'hash': '0x1c1909bf3

In [72]:
len(txes)

5141

In [80]:
fraction_fee = 0.03 / 100

In [82]:
total_fees = 0
for i in txes:
    total_fees += i['value'] * fraction_fee
    block_num = i['blockNum']
    block_end = int(block_num, 16)

print(f"Fees collected by {pool_addr} from block {block_start} to {block_end} is {round(total_fees, 2)}.")

Fees collected by 0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7 from block 13483245 to 13584561 is 1108278.2.


In [78]:
int(block_num, 16)

13483284